In [1]:
import csv

In [2]:
# вспомогательное

def parse_age(age_str: str) -> int | None:
    age_str = age_str.strip()

    if "," in age_str:
        return None

    try:
        age = int(age_str)
    except ValueError:
        return None

    if age <= 0:
        return None

    return age


def choose_year_word(age: int) -> str:
    # согласование года

    last_two = age % 100
    if 11 <= last_two <= 14:
        return "лет"

    last_digit = age % 10
    if last_digit == 1:
        return "год"
    if 2 <= last_digit <= 4:
        return "года"
    return "лет"

def get_gender_phrase(gender_str: str) -> tuple[str, str]:
    # согласование родов 
    gender = gender_str.strip().lower()

    if gender == "female":
        return "женского пола", "совершила"
    if gender == "male":
        return "мужского пола", "совершил"

    # на всякий случай
    return "неопределённого пола", "совершил"


def get_device_phrase(device_str: str, browser_str: str) -> str:
    device = device_str.strip().lower()
    browser = browser_str.strip()

    if device in ("mobile", "tablet"):
        return f"с мобильного устройства из браузера {browser}"

    if device in ("laptop", "desktop"):
        return f"с компьютера из браузера {browser}"

    # Если тип устройства неизвестен, пишем только про браузер
    return f"из браузера {browser}"

In [3]:
def build_description(row: list[str]) -> str | None:
    """
    порядок:
    0 - ФИО
    1 - устройство (mobile / desktop / ...)
    2 - браузер
    3 - пол (male / female)
    4 - возраст
    5 - сумма чека
    6 - регион
    """
    # на всякий случай
    if len(row) < 7:
        return None

    full_name = row[0].strip()
    device_str = row[1]
    browser_str = row[2]
    gender_str = row[3]
    age_str = row[4]
    sum_str = row[5].strip()
    region = row[6].strip()

    age = parse_age(age_str)
    if age is None:
        return None

    gender_phrase, verb = get_gender_phrase(gender_str)
    years_word = choose_year_word(age)
    device_phrase = get_device_phrase(device_str, browser_str)

    description = (
        f"Пользователь {full_name} {gender_phrase}, {age} {years_word} "
        f"{verb} покупку на {sum_str} у.е. {device_phrase}. "
        f"Регион, из которого совершалась покупка: {region}."
    )

    return description


In [4]:
def read_customers(path: str) -> list[list[str]]:
    rows: list[list[str]] = []

    with open(path, encoding="utf-8", newline="") as file:
        reader = csv.reader(file)
        next(reader, None)

        for row in reader:
            if not row:
                continue
            rows.append(row)

    return rows


def write_descriptions(path: str, descriptions: list[str]) -> None:
    with open(path, "w", encoding="utf-8") as file:
        for line in descriptions:
            file.write(line + "\n")

In [5]:
def process_customers(input_path: str, output_path: str) -> None:
    rows = read_customers(input_path)
    descriptions: list[str] = []

    for row in rows:
        description = build_description(row)
        if description is None:
            continue
        descriptions.append(description)

    write_descriptions(output_path, descriptions)


In [6]:
input_file = "web_clients_correct.csv"  
output_file = "output.txt"

process_customers(input_file, output_file)